In [1]:
import pandas as pd
import numpy as np

In [16]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import roc_auc_score, confusion_matrix

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [4]:
from sklearn.utils import resample

In [5]:
from lightgbm import LGBMClassifier

In [6]:
def full_path(path: str) -> str:
    path_to_project = 'C:\\Users\\Arseniy\\Documents\\Skillbox\\diploma_ML\\'
    return path_to_project + path 

In [7]:
df = pd.read_parquet(full_path('data/res_10.pq'))
df

,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,...,enc_loans_credit_type_3,enc_loans_credit_type_4,enc_loans_credit_type_5,enc_loans_credit_type_6,enc_loans_credit_type_7,rn_max,credit_history_length_max,num_loan_months_max,frac_loan_months_max,target
index,,,,,,,,,,,,,,,,,,,,,
0,9,10,10,10,10,6,9,9,1,2,...,2,7,0,0,0,10,23.0,1,0.083333,0
1,12,10,12,12,11,10,12,11,1,2,...,3,8,0,0,0,14,24.0,2,0.285714,0
2,3,2,2,2,3,1,3,2,2,2,...,2,1,0,0,0,3,23.0,8,0.521739,0
3,15,15,15,15,15,8,14,14,5,6,...,4,9,1,0,0,15,24.0,0,0.000000,0
4,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,1,0.0,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,9,5,11,11,11,8,11,7,2,5,...,4,6,0,0,1,11,24.0,5,0.320000,0
2999996,12,13,12,13,13,9,11,11,5,4,...,4,8,0,0,1,13,24.0,2,0.333333,0
2999997,9,7,10,10,10,6,10,9,1,1,...,3,7,0,0,0,10,24.0,6,0.545455,0


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='target'), df['target'], stratify=df.target, test_size=0.2, random_state=44)

In [9]:
X_train.shape

(2400000, 384)

In [10]:
y_test.shape

(600000,)

## LightGBM

In [23]:
lgbm = LGBMClassifier(class_weight='balanced', random_state=44)
lgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 85154, number of negative: 2314846
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 3.532378 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5790
[LightGBM] [Info] Number of data points in the train set: 2400000, number of used features: 372
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


LGBMClassifier(class_weight='balanced', random_state=44)

In [24]:
confusion_matrix(y_test, lgbm.predict(X_test))

array([[384354, 194358],
       [  6148,  15140]], dtype=int64)

In [25]:
roc_auc_score(y_test, lgbm.predict_proba(X_test)[:,1])

0.7555609440573365

In [26]:
temp = pd.DataFrame(lgbm.feature_importances_, index=X_train.columns, columns=['imp'])

In [27]:
for k, v in temp.sort_index().imp.items():
    print(f"{k}: {'-' if v else '0'}")

credit_history_length_max: -
enc_loans_account_holder_type_0: 0
enc_loans_account_holder_type_1: -
enc_loans_account_holder_type_2: 0
enc_loans_account_holder_type_3: -
enc_loans_account_holder_type_4: -
enc_loans_account_holder_type_5: 0
enc_loans_account_holder_type_6: -
enc_loans_credit_status_0: 0
enc_loans_credit_status_1: -
enc_loans_credit_status_2: -
enc_loans_credit_status_3: -
enc_loans_credit_status_4: -
enc_loans_credit_status_5: -
enc_loans_credit_status_6: 0
enc_loans_credit_type_0: -
enc_loans_credit_type_1: -
enc_loans_credit_type_2: -
enc_loans_credit_type_3: -
enc_loans_credit_type_4: -
enc_loans_credit_type_5: -
enc_loans_credit_type_6: -
enc_loans_credit_type_7: -
enc_paym_0_0: -
enc_paym_0_1: -
enc_paym_0_2: -
enc_paym_0_3: -
enc_paym_10_0: -
enc_paym_10_1: 0
enc_paym_10_2: -
enc_paym_10_3: 0
enc_paym_11_1: -
enc_paym_11_2: -
enc_paym_11_3: -
enc_paym_11_4: 0
enc_paym_12_0: -
enc_paym_12_1: -
enc_paym_12_2: -
enc_paym_12_3: 0
enc_paym_13_0: -
enc_paym_13_1: -
enc_p

In [28]:
temp.sort_values(by='imp', ascending=False).head(20)

,imp
pre_util_3,89
frac_loan_months_max,81
pre_till_pclose_10,76
enc_loans_credit_type_0,74
pre_util_6,73
pre_loans_outstanding_1,69
pre_loans_credit_cost_rate_9,57
pre_loans_credit_cost_rate_11,56
num_loan_months_max,48
pre_util_16,45


## Hist Gradient Boosting

In [20]:
hgb = HistGradientBoostingClassifier(class_weight='balanced', random_state=44)
hgb.fit(X_train, y_train)

HistGradientBoostingClassifier(class_weight='balanced', random_state=44)

In [21]:
confusion_matrix(y_test, hgb.predict(X_test))

array([[384402, 194310],
       [  6089,  15199]], dtype=int64)

In [22]:
roc_auc_score(y_test, hgb.predict_proba(X_test)[:,1])

0.7557017492405571

## HGB tuning

In [53]:
hgb = HistGradientBoostingClassifier(
    class_weight='balanced', 
    random_state=44, 
    scoring='roc_auc', 
    n_iter_no_change=50, 
    tol=1e-5)

In [81]:
gs = GridSearchCV(
    hgb,
    {
        # 'learning_rate': (0.1, 0.5, 1.0), # 0.1
        'l2_regularization': (20.0, 30.0), # 10
        # 'max_iter': (20, 100, 200), # 100
        # 'max_leaf_nodes': (10, 30, 100) # 30
        
    },
    scoring='roc_auc',
    cv=5,
    verbose=2,
    n_jobs=4,
    pre_dispatch='n_jobs'
)

In [82]:
gs.fit(data2, target[data2.index])

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(cv=5,
             estimator=HistGradientBoostingClassifier(class_weight='balanced',
                                                      n_iter_no_change=50,
                                                      random_state=44,
                                                      scoring='roc_auc',
                                                      tol=1e-05),
             n_jobs=4, param_grid={'l2_regularization': (20.0, 30.0)},
             pre_dispatch='n_jobs', scoring='roc_auc', verbose=2)

In [83]:
res = pd.concat((res, pd.DataFrame(gs.cv_results_)), ignore_index=True)

In [84]:
res.sort_values(by='mean_test_score', ascending=False)[['mean_fit_time', 'params', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score']]

,mean_fit_time,params,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score
18,182.809031,{'l2_regularization': 30.0},0.772052,0.766935,0.771760,0.761958,0.767454,0.003962
17,207.520160,{'l2_regularization': 20.0},0.771374,0.766976,0.773341,0.761402,0.767331,0.004516
16,184.316712,{'l2_regularization': 10.0},0.771287,0.766424,0.773452,0.761136,0.767029,0.004732
14,177.814388,{'l2_regularization': 3.0},0.770827,0.766197,0.770276,0.760970,0.766127,0.004001
15,214.158009,{'l2_regularization': 4.0},0.769373,0.765858,0.772170,0.760659,0.766009,0.004338
3,228.111624,{'l2_regularization': 1.0},0.770840,0.764593,0.771379,0.761264,0.765990,0.004332
13,207.803556,{'l2_regularization': 2.0},0.769764,0.765534,0.771879,0.760148,0.765913,0.004411
11,208.284054,{'max_leaf_nodes': 30},0.769656,0.765166,0.771262,0.760364,0.765826,0.004108
0,228.670399,{'l2_regularization': 0.0},0.769530,0.765899,0.772340,0.759486,0.765815,0.004741
8,201.737245,{'max_iter': 100},0.769530,0.765899,0.772340,0.759486,0.765815,0.004741


In [11]:
hgb = HistGradientBoostingClassifier(
    l2_regularization=10,
    class_weight='balanced', 
    scoring='roc_auc', 
    n_iter_no_change=50, 
    tol=1e-5,
    random_state=44, 
)

In [12]:
hgb.fit(X_train, y_train)

HistGradientBoostingClassifier(class_weight='balanced', l2_regularization=10,
                               n_iter_no_change=50, random_state=44,
                               scoring='roc_auc', tol=1e-05)

In [13]:
confusion_matrix(y_test, hgb.predict(X_test))

array([[384530, 194182],
       [  6088,  15200]], dtype=int64)

In [14]:
roc_auc_score(y_test, hgb.predict_proba(X_test)[:,1])

0.7557300154103052

In [18]:
cv = cross_validate(
    hgb,
    X_train,
    y_train,
    cv=4,
    scoring='roc_auc',
    verbose=2,
    return_estimator=True,
    n_jobs=1,
    pre_dispatch='n_jobs'
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 5.4min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.4min remaining:    0.0s


[CV] END .................................................... total time= 6.5min
[CV] END .................................................... total time= 6.1min
[CV] END .................................................... total time= 5.4min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 23.5min finished


In [19]:
cv

{'fit_time': array([321.26248407, 386.74835539, 364.46279168, 317.81277347]),
 'score_time': array([4.35203004, 5.84825563, 4.39252734, 4.51769066]),
 'estimator': [HistGradientBoostingClassifier(class_weight='balanced', l2_regularization=10,
                                 n_iter_no_change=50, random_state=44,
                                 scoring='roc_auc', tol=1e-05),
  HistGradientBoostingClassifier(class_weight='balanced', l2_regularization=10,
                                 n_iter_no_change=50, random_state=44,
                                 scoring='roc_auc', tol=1e-05),
  HistGradientBoostingClassifier(class_weight='balanced', l2_regularization=10,
                                 n_iter_no_change=50, random_state=44,
                                 scoring='roc_auc', tol=1e-05),
  HistGradientBoostingClassifier(class_weight='balanced', l2_regularization=10,
                                 n_iter_no_change=50, random_state=44,
                                 scoring=

In [88]:
cv['test_score'].mean()

0.7670292752388981